In [1]:
# import dependancies
import pandas as pd
import random
from datetime import datetime, timedelta
from yahoo_fin.stock_info import *
import yfinance as yf
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
import finnhub
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# change to your api keys 

#alphavantage_api_key = 'AJP4EKR4LE52NNK4'
alphavantage_api_key = 'LODFNW3D36E9Q39H'
polygon_api_key = 'J9UUfkyTylf3D_s76b5pcMCnLSaKTNpD'
finnhub_client = finnhub.Client(api_key="cldcmr9r01qn959k329gcldcmr9r01qn959k32a0")


### Screening US Mega & Large Cap Stocks

In [2]:
file_path = 'CSV_files/nasdaq_screener_1700335987448.csv'

# Read the CSV file into a Pandas DataFrame
stock_list_df = pd.read_csv(file_path)

# Display the DataFrame
stock_list_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$113.15,-1.04,-0.911%,3.310625e+10,United States,1999.0,2700770,Industrials,Electrical Products
1,AAPL,Apple Inc. Common Stock,$189.69,-0.02,-0.011%,2.950202e+12,United States,1980.0,50922657,Technology,Computer Manufacturing
2,ABBV,AbbVie Inc. Common Stock,$138.30,0.02,0.014%,2.441738e+11,United States,2012.0,4332297,Health Care,Biotechnology: Pharmaceutical Preparations
3,ABNB,Airbnb Inc. Class A Common Stock,$127.15,0.87,0.689%,8.244890e+10,United States,2020.0,3795704,Consumer Discretionary,Hotels/Resorts
4,ABT,Abbott Laboratories Common Stock,$99.55,-0.71,-0.708%,1.728246e+11,United States,NaN,6069070,Health Care,Biotechnology: Pharmaceutical Preparations


In [3]:
# create an array of tickers

ticker_list = stock_list_df['Symbol'].values

### Generating functions to retrieve stock information 

In [4]:
# define function to generate random date, and ensure that the date is a weekday, maybe start in 2014 or 2015

def generate_random_date(start_date, end_date):
    check_weekday = False
    while not check_weekday:
        # Convert the date strings to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.now()  # Using today's date as the end date

        # Calculate the range of days between the start and end date
        delta_days = (end_date_dt - start_date_dt).days

        # Generate a random number of days within the range
        random_days = random.randint(0, delta_days)

        # Calculate the random date
        random_date = start_date_dt + timedelta(days=random_days)

        # Check if the generated date is a weekday
        if random_date.weekday() < 5:
            check_weekday = True
            return random_date
        else:
            check_weekday = False



In [5]:
# define a function to choose a random stock from the list

def choose_random_stock(ticker_list):    
    
    # Randomly choose a stock from the list
    random_stock = random.choice(ticker_list)
    
    return random_stock.lower()
    

In [6]:
# define a function to check if data exists for stock on the date 

def check_stock_date(ticker, check_date):
    check_financials = False
    # Check if the input date is a Friday
    if check_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = check_date + timedelta(days=3)
    else:
        # Set end date to the next day
        adj_end_date = check_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    
    
    try:
        get_data(ticker, end_date=adj_end_date)
        get_stock_financials(ticker, check_date)
        

        check_financials = True
    except:
        check_financials = False
    
    if not check_financials:
        return False
    else:
        return True
    

In [7]:
# define a function to return stock industry

def return_stock_industry(symbol):
    symbol = symbol.upper()
    # Check if the symbol is in the DataFrame
    if symbol in stock_list_df['Symbol'].values:
        # Retrieve the sector for the given symbol
        sector = stock_list_df.loc[stock_list_df['Symbol'] == symbol, 'Sector'].iloc[0]
        return sector
    else:
        return f"Symbol '{symbol}' not found in the stock list."

In [8]:

# define a function to get the adjusted closing price of a stock on a particular day

def get_stock_price(ticker, stock_date):
    # Check if the input date is a Friday
    if stock_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        price_adj_end_date = stock_date + timedelta(days=3)
    else:
        # Set end date to the next day
        price_adj_end_date = stock_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=price_adj_end_date)
    # extracting the price 
    adj_close_price = stock_data['adjclose'].values[-1]
    
    return adj_close_price

In [9]:
# define a function to get stock trading volume

def get_stock_volume(ticker, vol_date):
    # Check if the input date is a Friday
    if vol_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        vol_adj_end_date = vol_date + timedelta(days=3)
    else:
        # Set end date to the next day
        vol_adj_end_date = vol_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=vol_adj_end_date)
    
    #extracting the volume
    trading_vol = stock_data['volume'].values[-1]
    
    return trading_vol

In [10]:
# function to get 4 previous weekdays from given_date

def get_previous_weekdays(weekday_date):
    previous_weekdays = []
    previous_date = weekday_date - timedelta(days=1)
    while len(previous_weekdays) < 4:
        if previous_date.weekday() < 5:
            previous_weekdays.append(previous_date)
        previous_date -= timedelta(days=1)

    return previous_weekdays


In [11]:
# function to get company's financial ratios by scraping stockanalysis.com

def get_stock_financials(ticker, fin_date):
    
    financial_ratios = {}

    # setting brwoser
    browser = Browser('chrome')
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        # Extract date columns excluding the first column
        date_columns = df.columns[2:-1]
        
        # Convert date columns to datetime objects
        date_columns = pd.to_datetime(date_columns)
        
        # Find the closest date column to the given date that is earlier
        earlier_dates = [date for date in date_columns if date < fin_date]
        closest_date_column = max(earlier_dates)

        # Access the closest column
        closest_column = closest_date_column.strftime('%Y-%m-%d')
        
    except:
        print("Table not found on the page.")
    
    

    #extracting financial ratios
    try: 
        financial_ratios['pe_ratio']=float(df.loc[df['Quarter Ended'] == 'PE Ratio', closest_column].values[0])
        financial_ratios['debt_to_equity']=float(df.loc[df['Quarter Ended'] == 'Debt / Equity Ratio', closest_column].values[0])
        financial_ratios['quick_ratio']=float(df.loc[df['Quarter Ended'] == 'Quick Ratio', closest_column].values[0])
        total_shareholder_return_str = df.loc[df['Quarter Ended'] == 'Total Shareholder Return', closest_column].values[0]
        financial_ratios['total_shareholder_return'] = float(total_shareholder_return_str.strip('%'))
    
    except:
        pass 
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table by class name
    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
    # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    except:
        print("Table not found on the page.")
    
    try:
        profit_margin_str= df.loc[df['Quarter Ended'] == 'Profit Margin', closest_column].values[0]
        financial_ratios['profit_margin'] = float(profit_margin_str.strip('%'))
        cash_margin_str= df.loc[df['Quarter Ended'] == 'Free Cash Flow Margin', closest_column].values[0]
        financial_ratios['free_cash_margin'] = float(cash_margin_str.strip('%'))
    except:
        pass

    return financial_ratios
    
    

In [12]:
# function to calculate stock's volatility

def calculate_volatility(ticker, sd_date):
    df=get_data(ticker, end_date=sd_date)
    # Calculate daily returns
    df['daily_return'] = df['adjclose'].pct_change()

    # Calculate volatility as the standard deviation of daily returns
    volatility = df['daily_return'].std()

    return volatility

In [13]:
#------------------------NEED TO FIND A NEW SOURCE---------------------------------

def get_news_sentiment(ticker, news_date):
    
    # Extract year, month, and day
    news_end_date=news_date
    end_year = news_end_date.year
    end_month = str(news_end_date.month).zfill(2)
    end_day = str(news_end_date.day).zfill(2)
    
    news_start_date = news_end_date - timedelta(weeks=4)
    start_year = news_start_date.year
    start_month = str(news_start_date.month).zfill(2)
    start_day = str(news_start_date.day).zfill(2)
   

In [14]:
# extracting daily federal interest rate information from alphavantage

# using alphavantage API
url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

daily_interest_rates = data

# converting json to dataframe
daily_interest_rate_df = pd.DataFrame(daily_interest_rates['data'])

# Export DataFrame to CSV so that we can refer to it without making an api call
daily_interest_rate_df.to_csv('CSV_files/daily_interest_rates.csv', index=False)


In [15]:
# extracting monthly CPI information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_cpi = data

# converting json to dataframe
monthly_cpi_df = pd.DataFrame(monthly_cpi['data'])
monthly_cpi_df['date'] = pd.to_datetime(monthly_cpi_df['date'])


# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_cpi_df.to_csv('CSV_files/monthly_cpi.csv', index=False)

In [16]:
# extracting monthly unemployment rate information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_unemployment_rate = data

# converting json to dataframe
monthly_unemployment_rate_df = pd.DataFrame(monthly_unemployment_rate['data'])
monthly_unemployment_rate_df['date'] = pd.to_datetime(monthly_unemployment_rate_df['date'])

# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_unemployment_rate_df.to_csv('CSV_files/monthly_unemployment_rate.csv', index=False)

In [17]:
# define a function to extract economic indicators for the given date

def get_economic_indicators(econ_date):
    
    economic_indicators_dict={}
    
    # convert datetime object to matching string 
    formatted_date = econ_date.strftime('%Y-%m-%d')
    
    # retrieve CPI value
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['cpi'] = float(monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])    
    
    # retrieve federal interest rate value
    economic_indicators_dict['interest_rate'] = float(daily_interest_rate_df.loc[daily_interest_rate_df['date'] == formatted_date, 'value'].iloc[0])

    # retrieve unemployement rate
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['unemployment_rate'] = float(monthly_unemployment_rate_df.loc[monthly_unemployment_rate_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])
    
    return economic_indicators_dict

In [18]:
# function to retrieve technical indicators

def get_technical_indicators(ticker, tech_date):
    
    technical_indicators_dict={}
    
    # changing date into UNIX format
    end_timestamp = int(tech_date.timestamp())
    
    # choosing a timestamp of one week earlier
    start_tech_date = tech_date - timedelta(weeks=1)
    
    # changing the date to UNIX format
    start_timestamp = int(start_tech_date.timestamp())
    
    # retrieving SMA 
    technical_indicators_dict['sma']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='sma', indicator_fields={"timeperiod": 5})['sma'][-1]
    
    #retrieving RSI
    technical_indicators_dict['rsi']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='rsi', indicator_fields={"timeperiod": 5})['rsi'][-1]

    # retrieving EMA 
    technical_indicators_dict['ema']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='ema', indicator_fields={"timeperiod": 5})['ema'][-1]
    
    return technical_indicators_dict

In [19]:
# function to create label value for data

def create_label(ticker, y_date):
    print(y_date)
    initial_price = get_stock_price(ticker, y_date)
    print(initial_price)
    
    # find the next weekday
    
    # check if date landed on friday and calculate the next weekday date
    if y_date.weekday() == 4:
        next_weekday_date = y_date + timedelta(days=3)
    else:
        next_weekday_date = y_date + timedelta(days=1)

    print(next_weekday_date)
    
    next_price = get_stock_price(ticker, next_weekday_date)
    
    print(next_price)
    
    if next_price >= initial_price:
        return 1 # 1 is class increase, in order to simplify the classification we put stayed the same in same category as increase
    else:
        return 0 # 0 is class decrease

### Running a loop to generate multiple rows of data for training and testing

In [104]:
page=12

In [105]:
# list to store dictionaries
list_dict = []
page += 1
# runs loop until list_dic [change no. 4 as desired]
while len(list_dict) < 5:

    # data_dict
    data_dict = {}
    
    # checking to see if generated ticker-date combo has available data
    checker = False
    
    # runs this loop until checker is true
    while not checker:

        ticker = choose_random_stock(ticker_list)
        input_date = generate_random_date('2015-01-01', '2023-10-31')

        checker = check_stock_date(ticker, input_date)

    # using function to return stock industry
    data_dict['industry']=return_stock_industry(ticker)

    # retrieve weekly returns
    date = input_date
    for i in range(1, 5):
        data_dict[f'wr{i}']=(get_stock_price(ticker, date) - get_stock_price(ticker, date - timedelta(weeks = 1)))/get_stock_price(ticker, date - timedelta(weeks = 1))
        date = date - timedelta(weeks=1)

    # retrieve last 5 working day volumes
    data_dict['vol1'] = get_stock_volume(ticker, date)
    for i, weekday in enumerate(get_previous_weekdays(input_date)):
        data_dict[f'vol{i+1}'] = get_stock_volume(ticker, weekday)

    # retrieve stock financials    
    for key, value in get_stock_financials(ticker, input_date).items():
        data_dict[key] = value
    
    # retrieve stock volatility
    data_dict['volatility'] = calculate_volatility(ticker, input_date)

    # retrieve economic indicators
    for key, value in get_economic_indicators(input_date).items():
        data_dict[key] = value

    # retrieve stocktechnical indicators    
    for key, value in get_technical_indicators(ticker, input_date).items():
        data_dict[key] = value

    # generate a label for row    
    data_dict['label'] = create_label(ticker, input_date)
        
    # append dictionary to list
    list_dict.append(data_dict)
    print(len(list_dict))

2016-12-12 00:00:00
115.29151153564453
2016-12-13 00:00:00
115.0266342163086
1
2020-10-23 00:00:00
315.6000061035156
2020-10-26 00:00:00
306.32000732421875
2
2017-12-26 00:00:00
144.06642150878906
2017-12-27 00:00:00
145.3836669921875
3
2022-12-29 00:00:00
25.90412712097168
2022-12-30 00:00:00
25.469655990600586
4
2022-06-16 00:00:00
54.359130859375
2022-06-17 00:00:00
55.09037399291992
5


In [96]:
data_dict

{'industry': 'Industrials', 'wr1': -0.10792956205525854}

In [97]:
print(ticker, input_date)

fox 2019-03-25 00:00:00


In [98]:
pd.to_datetime(input_date).to_period('M')

Period('2019-03', 'M')

In [99]:
monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(input_date).to_period('M'), 'value'].iloc[0]  

'254.202'

In [100]:
list_dict

[{'industry': 'Real Estate',
  'wr1': 0.02978894594018636,
  'wr2': 0.012986960090096145,
  'wr3': 0.01237440795733126,
  'wr4': 0.023752754330137467,
  'vol1': 3135700,
  'vol2': 2522800,
  'vol3': 3783400,
  'vol4': 2918300,
  'pe_ratio': 19.81,
  'debt_to_equity': 1.12,
  'profit_margin': 21.85,
  'free_cash_margin': 52.61,
  'volatility': 0.021052946936954822,
  'cpi': 237.336,
  'interest_rate': 0.12,
  'unemployment_rate': 5.1,
  'sma': 27.063333333333333,
  'rsi': 77.96976241900646,
  'ema': 27.089166666666664,
  'label': 0},
 {'industry': 'Energy',
  'wr1': 0.002052750982860902,
  'wr2': 0.037997180728882196,
  'wr3': -0.03421985382877391,
  'wr4': -0.037413696687608265,
  'vol1': 948300,
  'vol2': 798500,
  'vol3': 782300,
  'vol4': 1030600,
  'pe_ratio': -41.4,
  'debt_to_equity': 0.81,
  'quick_ratio': 0.63,
  'total_shareholder_return': 7.42,
  'profit_margin': 13.94,
  'free_cash_margin': 16.32,
  'volatility': 0.021310888341641892,
  'cpi': 267.054,
  'interest_rate': 0.0

### Export data into csv

In [101]:
page = 10

In [102]:
# remember to export data as csv
data_df = pd.DataFrame(list_dict)
data_df.to_csv(f'data_AP_{page}.csv', index=False)


In [103]:
data_df

,industry,wr1,wr2,wr3,wr4,vol1,vol2,vol3,vol4,pe_ratio,...,volatility,cpi,interest_rate,unemployment_rate,sma,rsi,ema,label,quick_ratio,total_shareholder_return
0,Real Estate,0.029789,0.012987,0.012374,0.023753,3135700,2522800,3783400,2918300,19.81,...,0.021053,237.336,0.12,5.1,27.063333,77.969762,27.089167,0,NaN,NaN
1,Energy,0.002053,0.037997,-0.034220,-0.037414,948300,798500,782300,1030600,-41.40,...,0.021311,267.054,0.07,6.1,29.316667,59.183673,29.280833,0,0.63,7.42
2,Technology,0.020308,-0.003445,0.017694,0.015464,3909600,3274300,3638300,3653200,-45.86,...,0.039442,242.839,0.66,4.7,71.026667,57.536606,70.954167,1,1.74,-22.74
3,Real Estate,-0.031250,0.079958,-0.039402,-0.040211,45500,49600,33300,26700,NaN,...,0.019074,297.711,3.83,3.6,20.430000,21.271077,20.368750,0,NaN,NaN
4,Consumer Discretionary,0.003619,-0.045443,-0.065325,-0.045519,6350200,7610700,10296300,8075600,10.89,...,0.030271,252.885,2.20,3.8,35.363333,61.142857,35.672917,0,NaN,NaN
5,Health Care,-0.018340,0.026481,0.012324,-0.026589,3375600,7995500,7344300,2764000,33.12,...,0.017580,277.948,0.08,4.2,79.690000,46.094699,80.067500,0,1.55,3.62
6,Technology,0.003328,0.021596,0.050820,-0.015216,186000,146900,113800,190700,93.50,...,0.032668,267.054,0.07,6.1,446.766667,52.327895,446.307917,1,3.17,-3.00
7,Technology,-0.030426,0.063138,-0.066828,0.004866,2338900,887000,1066800,845400,8.33,...,0.036349,296.808,2.33,3.5,59.006667,44.237918,59.258333,0,0.40,7.03
8,Technology,0.019518,0.002477,0.041935,0.036568,6485100,6240200,8040300,8196000,28.14,...,0.029338,303.363,4.83,3.4,95.616667,86.065574,95.400833,1,0.65,0.70
9,Finance,-0.032002,0.052768,-0.032972,-0.000784,2319900,1894500,1977000,3720000,22.13,...,0.025248,260.229,0.09,6.7,42.022189,25.706692,42.061494,0,0.31,1.86


In [21]:
page = 1

In [95]:
print(ticker, input_date)

psa 2018-03-14 00:00:00
